In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import os



In [42]:
df_orig = pd.read_csv(r"C:\Users\Pier\Documents\GitHub\Heart-Failure-ML\heart_failure_clinical_records_dataset.csv")

In [ ]:
df_orig.head()

In [43]:
props_binarias = ["anaemia","diabetes","high_blood_pressure","sex","smoking","DEATH_EVENT"]
props_continuas = ["age","creatinine_phosphokinase","ejection_fraction","platelets","serum_creatinine","serum_sodium"] #NO TIENE TIME PORQ ES PARA LOS GRAFICOS

In [44]:
from sklearn.preprocessing import MinMaxScaler
df_normalized = df_orig.copy() #SI LA QUEREMOS LLAMAR USAR "df_normalized"
to_normalize = props_continuas+["time"]
x = df_normalized[to_normalize].values
x_scaled = MinMaxScaler().fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=to_normalize, index = df_normalized.index)
df_normalized[to_normalize] = df_temp

##TENGO LA DATA LIMPIA Y AHORA?

#### TODO ESTO ES UN RESUMEN GUIA, NO SON LOS DATASETS Q VAMOS A USAR

### SEPARO EN TEST Y TRAIN




In [45]:
from sklearn.model_selection import train_test_split

feat_train, feat_test, targ_train, targ_test = train_test_split(df_normalized.drop(["DEATH_EVENT","time"],axis=1),df_normalized["DEATH_EVENT"],test_size=0.2)

## REGRESION LINEAL

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

grado = 1

modelo = Pipeline([["poly",PolynomialFeatures(degree=grado,include_bias=True)],["regressor",LinearRegression(fit_intercept=False)]])

modelo.fit(feat_train,targ_train)


Pipeline(steps=[('poly', PolynomialFeatures(degree=1)),
                ['regressor', LinearRegression(fit_intercept=False)]])

In [ ]:
modelo["regressor"].coef_

In [59]:
from sklearn.metrics import mean_squared_error as mse

np.sqrt(mse(targ_test,modelo.predict(feat_test)))

0.3743389714724081

In [ ]:
from sklearn.model_selection import GridSearchCV

hyperparam = {"features__degree":[n for n in range(1,6)]}
#CUANDO HAGO GRILLA PONGO FEATURES
modelo = Pipeline([["features",PolynomialFeatures(degree=grado,include_bias=True)],["regressor",LinearRegression(fit_intercept=False)]])
grilla = GridSearchCV(modelo,hyperparam,refit=True,cv=len(feat_train),scoring="neg_mean_squared_error",verbose=True,n_jobs=-1)
grilla.fit(feat_train,targ_train)


In [ ]:
print(grilla.best_params_)

In [ ]:
for ni, i in enumerate(grilla.cv_results_['params']):
  print(i['features__degree'],-grilla.cv_results_['mean_test_score'][ni])
plt.errorbar(grilla.cv_results_['param_features__degree'].data,np.sqrt(-grilla.cv_results_['mean_test_score']),yerr=grilla.cv_results_['std_test_score'])
plt.yscale('log')
plt.xlabel('grado')
plt.ylabel('RMSE en validacion')

In [ ]:
besthyper_model = grilla.best_estimator_

np.sqrt(mse(targ_test,besthyper_model.predict(feat_test)))

# PRUEBA REGRESION LOGISTICA

In [46]:
from sklearn.linear_model import LogisticRegression

In [109]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

grado = 5

modelo = Pipeline([["poly",PolynomialFeatures(degree=grado,include_bias=True)],["regressor",LogisticRegression(fit_intercept=False, tol=0.0000001, penalty="l2")]])

modelo.fit(feat_train,targ_train)

Pipeline(steps=[('poly', PolynomialFeatures(degree=5)),
                ['regressor',
                 LogisticRegression(fit_intercept=False, tol=1e-07)]])

In [100]:
modelo["regressor"].coef_

array([[-0.33142943, -0.00957463,  0.04674713, ..., -0.19134939,
        -0.19134939, -0.01347039]])

In [110]:

from sklearn.metrics import accuracy_score

print(accuracy_score(targ_test, modelo.predict(feat_test)))

0.6333333333333333


In [111]:
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(targ_test, modelo.predict(feat_test))
print(cf)

[[32  9]
 [13  6]]


In [94]:
from sklearn.model_selection import GridSearchCV

hyperparam = {"poly__degree":[n for n in range(1,8)]}
#CUANDO HAGO GRILLA PONGO FEATURES
modelo = Pipeline([["poly",PolynomialFeatures(degree=grado,include_bias=True)],["regressor",LogisticRegression(fit_intercept=False, tol=0.0000001, penalty="l2")]])
grilla = GridSearchCV(modelo,hyperparam,refit=True,cv=100,scoring="f1",verbose=True,n_jobs=-1)
grilla.fit(feat_train,targ_train)

Fitting 100 folds for each of 7 candidates, totalling 700 fits


C:\Users\Pier\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 77 members, which is less than n_splits=100.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=100,
             estimator=Pipeline(steps=[['poly', PolynomialFeatures(degree=4)],
                                       ['regressor',
                                        LogisticRegression(fit_intercept=False,
                                                           tol=1e-07)]]),
             n_jobs=-1, param_grid={'poly__degree': [1, 2, 3, 4, 5, 6, 7]},
             scoring='f1', verbose=True)

In [95]:
print(grilla.best_params_)

{'poly__degree': 5}


In [97]:
print(grilla.best_score_)

0.27833333333333327


In [98]:
modelo = grilla.best_estimator_
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(targ_train, modelo.predict(feat_train))
print(cf)

[[155   7]
 [ 22  55]]
